In [1]:
#hide image in image
from PIL import Image
import csv

def int_to_bin(value):
    return format(value, '08b')

def embed_bits(carrier_pixel, bits):
    r, g, b = carrier_pixel
    r = (r & 0b11111100) | int(bits[0:2], 2)
    g = (g & 0b11111100) | int(bits[2:4], 2)
    b = (b & 0b11111100) | int(bits[4:6], 2)
    return (r, g, b)

def encode_image(carrier, passenger):
    carrier_pixels = carrier.load()
    passenger_pixels = passenger.load()

    width, height = passenger.size
    carrier_index = 0

    for y in range(height):
        for x in range(width):
            r, g, b = passenger_pixels[x, y]
            r_bits = int_to_bin(r)
            g_bits = int_to_bin(g)
            b_bits = int_to_bin(b)

            # Embed each 2 bits into 4 carrier pixels
            carrier_pixels[carrier_index % carrier.width, carrier_index // carrier.width] = embed_bits(carrier_pixels[carrier_index % carrier.width, carrier_index // carrier.width], r_bits[:2]+g_bits[:2]+b_bits[:2])
            carrier_index += 1

            carrier_pixels[carrier_index % carrier.width, carrier_index // carrier.width] = embed_bits(carrier_pixels[carrier_index % carrier.width, carrier_index // carrier.width], r_bits[2:4]+g_bits[2:4]+b_bits[2:4])
            carrier_index += 1

            carrier_pixels[carrier_index % carrier.width, carrier_index // carrier.width] = embed_bits(carrier_pixels[carrier_index % carrier.width, carrier_index // carrier.width], r_bits[4:6]+g_bits[4:6]+b_bits[4:6])
            carrier_index += 1

            carrier_pixels[carrier_index % carrier.width, carrier_index // carrier.width] = embed_bits(carrier_pixels[carrier_index % carrier.width, carrier_index // carrier.width], r_bits[6:]+g_bits[6:]+b_bits[6:])
            carrier_index += 1

    carrier.save('/content/encoded_image.png')



carrier_url = '/content/carrier.jpg'
passenger_url = '/content/small_passenger.jpg'
final = '/content/final.jpg'
log_f = '/content/log1.csv'
carrier = Image.open(carrier_url)
width1, height1 = carrier.size

passenger = Image.open(passenger_url)
width2, height2 = passenger.size

print(width1, height1)
print(width2, height2)

if width1 >= 4*width2 and height1 >= 4*height2:
    print("Carrier image is large enough to contain the passenger image.")
    encode_image(carrier, passenger)
else:
    print("Carrier image is too small to contain the passenger image.")

4840 2766
227 222
Carrier image is large enough to contain the passenger image.


In [2]:
#extract image from image
from PIL import Image

def bin_to_int(binary_str):
    return int(binary_str, 2)


def extract_bits(carrier_pixel):
    r, g, b = carrier_pixel
    r_bits = format(r, '08b')
    g_bits = format(g, '08b')
    b_bits = format(b, '08b')
    return r_bits[-2:], g_bits[-2:], b_bits[-2:]

def decode_image(carrier, width, height):
    carrier_pixels = carrier.load()
    decoded_image = Image.new('RGB', (width, height))
    decoded_pixels = decoded_image.load()

    carrier_index = 0

    for y in range(height):
        for x in range(width):
            r_bits = ""
            g_bits = ""
            b_bits = ""

            for _ in range(4):
                r, g, b = extract_bits(carrier_pixels[carrier_index % carrier.width, carrier_index // carrier.width])
                r_bits += r
                g_bits += g
                b_bits += b
                carrier_index += 1
            r = bin_to_int(r_bits)
            g = bin_to_int(g_bits)
            b = bin_to_int(b_bits)

            decoded_pixels[x, y] = (r, g, b)

    decoded_image.save('/content/decoded_image.png')

# Example Usage
carrier_url = '/content/encoded_image.png'
carrier = Image.open(carrier_url)

width, height = 227, 222

decode_image(carrier, width, height)


In [3]:
#image in video
import cv2
import numpy as np

def encode_image_into_video(video_path, image_path, output_video_path):
    cap = cv2.VideoCapture(video_path)
    image = cv2.imread(image_path)
    #print(image)
    img_height, img_width, _ = image.shape
    print(img_height, img_width)
    if not cap.isOpened():
        print("Error opening video file")
        return

    ret, frame = cap.read()
    if not ret or img_height > frame.shape[0] or img_width > frame.shape[1]:
        print("Image is too large to be hidden in the video.")
        return

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, cap.get(cv2.CAP_PROP_FPS),
                          (int(cap.get(3)), int(cap.get(4))))

    while cap.isOpened():
        ret, frame = cap.read()
        #print(cap.get(cv2.CAP_PROP_POS_FRAMES))
        if not ret:
            break

        if cap.get(cv2.CAP_PROP_POS_FRAMES) == 2:
            #print(frame[0:img_height, 0:img_width])
            frame[0:img_height, 0:img_width] = image

            #print('check2')
            #print(frame[0:img_height, 0:img_width])


        out.write(frame)

    cap.release()
    out.release()
    print("Image encoded successfully")


def decode_image_from_video(video_path, img_height, img_width):
    cap = cv2.VideoCapture(video_path)

    ret, frame = cap.read()
    if not ret:
        print("Failed to read the video.")
        return
    #print(frame[0:img_height, 0:img_width])
    hidden_image = frame[0:img_height, 0:img_width]

    cap.release()
    return hidden_image

video_path = '/content/video.mp4'
image_path = '/content/passenger2.png'
output_video_path = '/content/encoded_video.mp4'
encode_image_into_video(video_path, image_path, output_video_path)

decoded_image = decode_image_from_video(output_video_path, 400, 399)
cv2.imwrite('/content/decoded_image_from_video.png', decoded_image)

400 399
Image encoded successfully


True